In [9]:
import pandas as pd

In [11]:
# Cell produces a dict where the key value represents a specific week at a specific facility eg."2023-01-01-F001"
# The value is a tuple that holds the list of demands for a specific product over that week along with the facility id and capacity
optimazation_args_dict = {}
df_demand = pd.read_csv("../../demand.csv")
tmp = {"F001": 10741, "F002": 14425, "F003": 10536}
for _, row in df_demand.iterrows():
    key_value = row['date'] + '-' + row['facility_id']
    if key_value not in optimazation_args_dict:
        # Creating fist key entry for a specified week and associated faciltiy along with demand list and facility number and capacity
        optimazation_args_dict[key_value] = ([row['demand']], row['facility_id'] ,tmp[row['facility_id']])
    else:
        optimazation_args_dict[key_value][0].append(row['demand'])
print(optimazation_args_dict)

{'2023-01-01-F001': ([459, 702, 128, 230, 78, 84, 111, 93, 264, 214, 85, 527, 101, 72, 112, 48, 505, 403, 264, 62], 'F001', 10741), '2023-01-08-F001': ([662, 662, 165, 239, 123, 120, 130, 107, 433, 192, 112, 846, 102, 57, 102, 98, 450, 472, 253, 126], 'F001', 10741), '2023-01-15-F001': ([561, 695, 106, 255, 136, 135, 117, 139, 529, 223, 114, 489, 83, 86, 198, 89, 662, 530, 191, 114], 'F001', 10741), '2023-01-22-F001': ([864, 863, 154, 304, 106, 118, 114, 134, 428, 300, 102, 1149, 141, 86, 196, 86, 560, 565, 243, 137], 'F001', 10741), '2023-01-29-F001': ([672, 848, 126, 283, 107, 175, 181, 125, 462, 246, 108, 889, 197, 57, 129, 95, 643, 526, 284, 130], 'F001', 10741), '2023-02-05-F001': ([505, 694, 122, 345, 104, 156, 115, 135, 451, 229, 92, 739, 156, 64, 169, 97, 668, 520, 293, 102], 'F001', 10741), '2023-02-12-F001': ([435, 613, 106, 122, 76, 122, 97, 135, 358, 185, 114, 628, 128, 79, 125, 83, 546, 581, 255, 109], 'F001', 10741), '2023-02-19-F001': ([518, 608, 85, 262, 72, 115, 53, 10

In [ ]:
p = [4, 1, 6, 5]
v = [3, 2, 1, 3]
w = 10

dp_table = [[0] * (10 + 1) for _ in range(len(p) + 1)]
c_w = 0

for i in range(len(p)):
    if i == 0:
        dp_table[0][0] = 0
    else:
        if p[i-1] > w:
            c_w = c_w
        elif (c_w + p[i-1]) <= w:
            c_w += p[i-1]
            
            
            dp_table[i][c_w] = (p[i-1] * v[i-1]) + dp_table[i-1][max([0] + [a for a in p[::-1][len(p) - (i-1):]])]
        elif (c_w + p[i+1]) > w:
            c_w = p[i -1]
            elem = max([a for a in p[::-1][i-1:]])
            max_i = p.index(elem)
            
            # First check Data table for any value 
            #            
            
            dp_table[i][elem + c_w] = (p[i-1] * v[i-1]) + dp_table[max_i + 1][elem]
            
    